In [ ]:
from pathlib import Path
import re

# import mantid.simpleapi as mantid
import numpy as np
import scipp as sc
import scipp.constants
import scippneutron as scn
# import scippnexus as snx

In [ ]:
%matplotlib widget

In [ ]:
DATA_DIR = Path(".").resolve().parent / "data"

In [ ]:
raw = scn.load_with_mantid(DATA_DIR / "raw" / "MAP51007.raw")
raw_data = raw['data']
run_title = raw['run_title'].value

In [ ]:
# raw_data = raw_data['spectrum', :100].copy()

In [ ]:
m = re.search(r'Ei=([\d.+\-]+)(\w+)', run_title)
incident_energy = sc.scalar(float(m[1]), unit=m[2])

In [ ]:
m = re.search(r'Rot=([\d.+\-]+)', run_title)
sample_rotation = sc.spatial.rotations_from_rotvecs(sc.vector([0,float(m[1]),0], unit='deg'))

In [ ]:
ub_matrix = sc.spatial.linear_transform(value=[[0.2039,0.1019,0.0],[0.0,0.1754,0.0],[0.0,0.0,0.222]])

In [ ]:
raw_data.coords['incident_energy'] = incident_energy  # The loaded data has 0 here for some reason
raw_data.coords['sample_rotation'] = sample_rotation
raw_data.coords['ub_matrix'] = ub_matrix
# raw_data

In [ ]:
raw_data.sum('spectrum').plot()

In [ ]:
def hkl_elements_from_hkl_vec(*, hkl_vec):
    # fix for https://github.com/scipp/scippneutron/issues/580
    return {'h': hkl_vec.fields.x, 'k': hkl_vec.fields.y, 'l': hkl_vec.fields.z}

def wavelength_from_energy(E):
    return sc.constants.h / sc.sqrt(2 * sc.constants.m_n * E)


def inelastic_Q(*, incident_beam, scattered_beam, energy_transfer, incident_energy):
    v_i = incident_beam / sc.norm(incident_beam)
    v_f = scattered_beam / sc.norm(scattered_beam)

    E_f = incident_energy - energy_transfer
    k_f = 2*np.pi / wavelength_from_energy(E_f) * v_f
    k_i = 2*np.pi / wavelength_from_energy(incident_energy) * v_i

    Q = k_i - k_f
    return Q


def coord_transform_graph():
    from scippneutron.conversion.graph import tof, beamline
    from scippneutron.conversion import tof as _kernels
    return {
        **beamline.beamline(scatter=True),
        **tof.direct_inelastic(start='tof'),
        'hkl_vec': _kernels.hkl_vec_from_Q_vec,
        ('h', 'k', 'l'): hkl_elements_from_hkl_vec,
        'ub_matrix': _kernels.ub_matrix_from_u_and_b,
        'Q_vec': inelastic_Q,
        # 'Q_vec': _kernels.Q_vec_from_Q_elements,
    }


graph = coord_transform_graph()

In [ ]:
# sc.show_graph(graph)

In [ ]:
hkl_data = raw_data.transform_coords(['h', 'k', 'l', 'energy_transfer'],
                                     graph,
                                     keep_inputs=False, keep_intermediate=False)

In [ ]:
hkl_data

In [ ]:
for k in 'hkl':
    hkl_data.coords[k] = sc.midpoints(hkl_data.coords[k], dim='hkl_vec').to(unit='1/Å')
hkl_data.coords['energy_transfer'] = sc.midpoints(hkl_data.coords['energy_transfer'], dim='hkl_vec').to(unit='meV')

In [ ]:
u = hkl_data.coords['h'].unit
h_edges = sc.linspace('h', -0.2, 0.2, 201, unit=u)
l_edges = sc.linspace('l', -2, 2, 201, unit=u)
e_edges = sc.linspace('energy_transfer', 15.0, 20, 2, unit='meV')


hist = hkl_data.hist(l=l_edges, h=h_edges, energy_transfer=e_edges).sum('energy_transfer').plot(norm='log')
hist